In [1]:
import datetime as dt
import time
import random
import logging

from optibook.synchronous_client import Exchange
from libs import print_positions_and_pnl, round_down_to_tick, round_up_to_tick

from IPython.display import clear_output

logging.getLogger('client').setLevel('ERROR')    

In [2]:
def insert_quotes(exchange, instrument, bid_price, ask_price, bid_volume, ask_volume):
    if bid_volume > 0:
        # Insert new bid limit order on the market
        exchange.insert_order(
            instrument_id=instrument,
            price=bid_price,
            volume=bid_volume,
            side='bid',
            order_type='limit',
        )
        
        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

    if ask_volume > 0:
        # Insert new ask limit order on the market
        exchange.insert_order(
            instrument_id=instrument,
            price=ask_price,
            volume=ask_volume,
            side='ask',
            order_type='limit',
        )

        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

In [3]:
from transformers import pipeline
import re


sentiment_pipeline = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis", device = -1)
company_pipe = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")
MIN_SELLING_PRICE = 0.01
MAX_BUYING_PRICE = 10000

2023-11-19 07:22:21,369 [urllib3.connectionpool] [MainThread  ] Starting new HTTPS connection (1): huggingface.co:443
2023-11-19 07:22:21,490 [urllib3.connectionpool] [MainThread  ] https://huggingface.co:443 "HEAD /mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis/resolve/main/config.json HTTP/1.1" 200 0
2023-11-19 07:22:21,546 [urllib3.connectionpool] [MainThread  ] Starting new HTTPS connection (1): huggingface.co:443
2023-11-19 07:22:21,667 [urllib3.connectionpool] [MainThread  ] https://huggingface.co:443 "HEAD /mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis/resolve/main/config.json HTTP/1.1" 200 0
2023-11-19 07:22:21,728 [urllib3.connectionpool] [MainThread  ] Starting new HTTPS connection (1): huggingface.co:443
2023-11-19 07:22:21,854 [urllib3.connectionpool] [MainThread  ] https://huggingface.co:443 "HEAD /mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
2023-11-19 07:22:24,195 [

In [4]:
class asset():
    def __init__(self, name, volume_dual=0, volume_sentiment=0, time=0, active=False, t_dual=None, t_sentiment=None, algorithm_dual=False, algorithm_sentiment=False):
        self.volume_dual = volume_dual
        self.volume_sentiment = volume_sentiment
        self.time = time
        self.active = active
        self.name_A = name
        self.name_B = f"{name}_B"
        self.type_dual = t_dual
        self.type_sentiment = t_sentiment
        self.algorithm_dual = algorithm_dual
        self.algorithm_sentiment = algorithm_sentiment
        
names = ["CSCO", "ING", "NVDA", "PFE", "SAN"]

assets = [asset(i) for i in names]
print(assets)

[<__main__.asset object at 0x7fc54fbb4898>, <__main__.asset object at 0x7fc54fbb4860>, <__main__.asset object at 0x7fc54fbb4940>, <__main__.asset object at 0x7fc54fbb49b0>, <__main__.asset object at 0x7fc54fbb4a20>]


In [5]:
def sell_all(instrument_id):
    positions = exchange.get_positions()
    pos = positions[instrument_id]
    
    if pos > 0:
        print(f'-- Inserting sell order for {pos} lots of {instrument_id}, with limit price {MIN_SELLING_PRICE:.2f}')
        exchange.insert_order(instrument_id, price=MIN_SELLING_PRICE, volume=pos, side='ask', order_type='ioc')
    elif pos < 0:
        print(f'-- Inserting buy order for {abs(pos)} lots of {instrument_id}, with limit price {MAX_BUYING_PRICE:.2f}')
        exchange.insert_order(instrument_id, price=MAX_BUYING_PRICE, volume=-pos, side='bid', order_type='ioc')
    else:
        print(f'-- No initial position in {instrument_id}, skipping..')
        
    time.sleep(0.10) 
    
TOPICS = ['Cisco', 'ING', 'Nvidia', 'Pfizer', 'Santander']


def predict_sentiment(text):
    """predict sentiment for a sentence"""
    result = sentiment_pipeline(text)
    score = result[0]['score']
    sentiment = result[0]['label']
    return [sentiment, score]


def preprocess(tweet):
    """preprocess to drop the author @ and hashtags #"""
    pre_tweet = re.sub(r'#\w+', '', tweet)
    processed_tweet = re.sub(r'@\w+', '', pre_tweet)
    return processed_tweet


def get_topic(text):
    """predict main topic for a sentence"""
    result = company_pipe(text, TOPICS)
    score = result["scores"][0]
    label = result["labels"][0]
    return [label, score]


mapping_of_topics = {"Pfizer": "PFE", "ING": "ING", "Nvidia": "NVDA", "Santander": "SAN", "Cisco": "CSCO"}
def map_topic(topic):
    return mapping_of_topics[topic]


def run_pipe(text, assets):
    #preprocessed_text = preprocess(text)

    topic, topic_score = get_topic(text)
    print(topic_score)
    if topic_score > 0.9:
        company = map_topic(topic)
        print(company)
        
        # process on two markets for company  
    
        for instrument_id in [company, company+"_B"]:
            exchange.delete_orders(instrument_id)
            sell_all(instrument_id)
            for a in assets:
                if a.name_A == instrument_id:
                    a.active = False
                    a.algorithm_dual = False
                    a.algorithm_sentiment = False
        #            time.sleep(20)
        
        instrument_id = company
        
        sentiment, sentiment_score = predict_sentiment(text)
        print(sentiment_score)
        
        position = exchange.get_positions()[company]
        max_volume_buy = POSITION_LIMIT - position
        max_volume_sell = POSITION_LIMIT + position

        max_volume = min(max_volume_buy, max_volume_sell, int(100*sentiment_score))
        if sentiment == "positive":
            exchange.delete_orders(instrument_id)
            instrument_order_book = exchange.get_last_price_book(instrument_id)
            position = exchange.get_positions()[instrument_id]
            best_bid_price = instrument_order_book.bids[0].price
            exchange.insert_order(instrument_id, price=best_bid_price+10, volume=max_volume, side='bid', order_type='limit')
            
            for a in assets:
                if a.name_A == instrument_id:
                    a.active = True
                    a.time = dt.datetime.now()
                    a.type_sentiment = 1
                    a.algorithm_sentiment = True
                    a.volume_sentiment = max_volume
                    break
            
        elif sentiment == "negative":
            exchange.delete_orders(instrument_id)
            instrument_order_book = exchange.get_last_price_book(instrument_id)
            position = exchange.get_positions()[instrument_id]
            best_ask_price = instrument_order_book.asks[0].price
            exchange.insert_order(instrument_id, price=best_ask_price-10, volume=max_volume, side='ask', order_type='limit')
            for a in assets:
                if a.name_A == instrument_id:
                    a.active = True
                    a.time = dt.datetime.now()
                    a.type_sentiment = 2
                    a.algorithm_sentiment = True
                    a.volume_sentiment = max_volume
                    break
    
    for a in assets:
        if a.algorithm_sentiment:
            if (dt.datetime.now() - a.time).total_seconds() >= 40:
            
                if a.type_sentiment == 1:
                    instrument_order_book = exchange.get_last_price_book(a.name_A)
                    position = exchange.get_positions()[a.name_A]
                    best_ask_price = instrument_order_book.asks[0].price
                    exchange.insert_order(a.name_A, price=best_ask_price-10, volume=a.volume_sentiment, side='ask', order_type='limit')
                    a.active = False
                    a.algorithm_sentiment = False
                else:
                    instrument_order_book = exchange.get_last_price_book(a.name_A)
                    position = exchange.get_positions()[a.name_A]
                    best_bid_price = instrument_order_book.bids[0].price
                    exchange.insert_order(a.name_A, price=best_bid_price+10, volume=a.volume_sentiment, side='bid', order_type='limit')
                    a.active = False
                    a.algorithm_sentiment = False
                          
    
    return assets
    
def check_news(assets):
    news = exchange.poll_new_social_media_feeds()
    for i in news: 
        text = i.post
        print(text)
        assets = run_pipe(text, assets)
        print()
    return assets

In [ ]:
exchange = Exchange()
exchange.connect()

INSTRUMENTS = exchange.get_instruments()

QUOTED_VOLUME = 10
FIXED_MINIMUM_CREDIT = 0.15
PRICE_RETREAT_PER_LOT = 0.005
POSITION_LIMIT = 100

while True:
    print(f'')
    print(f'-----------------------------------------------------------------')
    print(f'TRADE LOOP ITERATION ENTERED AT {str(dt.datetime.now()):18s} UTC.')
    print(f'-----------------------------------------------------------------')

    # Display our own current positions in all stocks, and our PnL so far
    print_positions_and_pnl(exchange)
    print(f'')
    print(f'          (ourbid) mktbid :: mktask (ourask)')
    
    assets = check_news(assets)
    
    for a in assets: # INSTRUMENTS.values():
        instrument_id_1 = a.name_A
        instrument_id_2 = a.name_B
        
        time.sleep(0.2)
        exchange.delete_orders(instrument_id_1) # TODO: check at home if there is another way with the booleans
        exchange.delete_orders(instrument_id_2)
        
        instrument_order_book_1 = exchange.get_last_price_book(instrument_id_1)
        instrument_order_book_2 = exchange.get_last_price_book(instrument_id_2)
        
        if not (instrument_order_book_1 and instrument_order_book_1.bids and instrument_order_book_1.asks):
                print(f'{instrument_id_1:>6s} --     INCOMPLETE ORDER BOOK')
                continue
        if not (instrument_order_book_2 and instrument_order_book_2.bids and instrument_order_book_2.asks):
                print(f'{instrument_id_2:>6s} --     INCOMPLETE ORDER BOOK')
                continue
        
        if a.algorithm_dual:
                
            if a.type_dual == 1:
                if instrument_order_book_1.bids[0].price <= instrument_order_book_2.asks[0].price or (dt.datetime.now() - a.time).total_seconds() >= 50:
                    print(exchange.get_pnl())
                    print("    --> INSIDE 1:, ", instrument_order_book_1.bids[0].price,  instrument_order_book_2.asks[0].price)
                    exchange.insert_order(instrument_id_2, price=instrument_order_book_1.bids[0].price - 5, # instrument_order_book_1.bids[0].price * 0.05, 
                                          volume=a.volume_dual, side='ask', order_type='limit')
                    exchange.insert_order(instrument_id_1, price=instrument_order_book_2.asks[0].price + 5, # instrument_order_book_2.asks[0].price * 0.05, 
                                          volume=a.volume_dual, side='bid', order_type='limit')
                    time.sleep(0.1)
                    
                    a.active = False
                    a.algorithm_dual = False
                    print(exchange.get_pnl())
                
            elif a.type_dual == 2:
                if instrument_order_book_2.bids[0].price <= instrument_order_book_1.asks[0].price or (dt.datetime.now() - a.time).total_seconds() >= 50:
                    pnl = exchange.get_pnl()
                    print("    --> INSIDE 2:, ", instrument_order_book_2.bids[0].price,  instrument_order_book_1.asks[0].price)
                    exchange.insert_order(instrument_id_1, price=instrument_order_book_2.bids[0].price - 5, # instrument_order_book_2.bids[0].price * 0.05, 
                                          volume=a.volume_dual, side='ask', order_type='limit')
                    exchange.insert_order(instrument_id_2, price=instrument_order_book_1.asks[0].price + 5, # instrument_order_book_1.asks[0].price * 0.05, 
                                          volume=a.volume_dual, side='bid', order_type='limit')    
                    time.sleep(0.1)
                    
                    a.active = False
                    a.algorithm_dual = False
                    print(exchange.get_pnl())
            
        else:
            print("CHECKING")
            position_1 = exchange.get_positions()[instrument_id_1]
            position_2 = exchange.get_positions()[instrument_id_2]

            best_ask_price_1 = instrument_order_book_1.asks[0].price
            best_ask_price_2 = instrument_order_book_2.asks[0].price
            best_bid_price_1 = instrument_order_book_1.bids[0].price
            best_bid_price_2 = instrument_order_book_2.bids[0].price

            # TODO: fix it taking into account all cases separately
            max_volume_buy = min(POSITION_LIMIT - position_1, POSITION_LIMIT - position_2)
            max_volume_sell = min(POSITION_LIMIT + position_1, POSITION_LIMIT + position_2)

            max_volume = min(max_volume_buy, max_volume_sell, 50)
            #count_sec = 0

            if best_bid_price_1 > (best_ask_price_2 + 1):
                print("Type 1:, pnl:", exchange.get_pnl())
                a.time = dt.datetime.now()
                print(best_bid_price_1, best_ask_price_2)
                
                try:
                    exchange.insert_order(instrument_id_1, price=best_ask_price_2-5, volume=max_volume, side='ask', order_type='limit')
                    exchange.insert_order(instrument_id_2, price=best_bid_price_1+5, volume=max_volume, side='bid', order_type='limit')

                    time.sleep(0.1)

                    a.active = True
                    a.volume_dual = max_volume
                    a.type_dual = 1
                    a.algorithm_dual = True
                except:
                    pass
            


            elif best_bid_price_2 > (best_ask_price_1 + 1):
                print("Type 2:, pnl:", exchange.get_pnl())
                a.time = dt.datetime.now()
                print(best_bid_price_2, best_ask_price_1)
                try:
                    exchange.insert_order(instrument_id_2, price=best_ask_price_1-5, volume=max_volume, side='ask', order_type='limit')
                    exchange.insert_order(instrument_id_1, price=best_bid_price_2+5, volume=max_volume, side='bid', order_type='limit')

                    time.sleep(0.1)

                    a.active = True
                    a.volume_dual = max_volume
                    a.type_dual = 2
                    a.algorithm_dual = True
                except:
                    pass
                
        if not a.algorithm_dual and not a.algorithm_sentiment:
        
            for instrument in [a.name_A]: # , a.name_B]:
                exchange.delete_orders(instrument)

                # Obtain order book and only skip this instrument if there are no bids or offers available at all on that instrument,
                # as we we want to use the mid price to determine our own quoted price
                instrument_order_book = exchange.get_last_price_book(instrument)
                if not (instrument_order_book and instrument_order_book.bids and instrument_order_book.asks):
                    print(f'{instrument:>6s} --     INCOMPLETE ORDER BOOK')
                    continue

                # Obtain own current position in instrument
                position = exchange.get_positions()[instrument]

                # Obtain best bid and ask prices from order book to determine mid price
                best_bid_price = instrument_order_book.bids[0].price
                best_ask_price = instrument_order_book.asks[0].price
                mid_price = (best_bid_price + best_ask_price) / 2.0 

                # Calculate our fair/theoretical price based on the market mid price and our current position
                theoretical_price = mid_price - PRICE_RETREAT_PER_LOT * position

                # Calculate final bid and ask prices to insert
                bid_price = round_down_to_tick(theoretical_price - FIXED_MINIMUM_CREDIT, 0.1 )# exchange.tick_size(instrument)) #instrument.tick_size)
                ask_price = round_up_to_tick(theoretical_price + FIXED_MINIMUM_CREDIT, 0.1) #exchange.tick_size(instrument)) #instrument.tick_size)

                # Calculate bid and ask volumes to insert, taking into account the exchange position_limit
                max_volume_to_buy = POSITION_LIMIT - position
                max_volume_to_sell = POSITION_LIMIT + position

                bid_volume = min(QUOTED_VOLUME, max_volume_to_buy)
                ask_volume = min(QUOTED_VOLUME, max_volume_to_sell)

                # Display information for tracking the algorithm's actions
                print(f'{instrument:>6s} -- ({bid_price:>6.2f}) {best_bid_price:>6.2f} :: {best_ask_price:>6.2f} ({ask_price:>6.2f})')

                # Insert new quotes
                insert_quotes(exchange, instrument, bid_price, ask_price, bid_volume, ask_volume)
        
        
    # Wait for a few seconds to refresh the quotes
    print(f'\nWaiting for 2 seconds.')
    time.sleep(0.2)
    
    # Clear the displayed information after waiting
    clear_output(wait=True)


-----------------------------------------------------------------
TRADE LOOP ITERATION ENTERED AT 2023-11-19 08:06:32.246835 UTC.
-----------------------------------------------------------------
Positions:
  NVDA      :  -45
  ING       :   10
  SAN       :  -15
  PFE       :  -10
  NVDA_B    :    0
  CSCO      :   -5
  ING_B     :    0
  PFE_B     :    0
  SAN_B     :    0
  CSCO_B    :    0

PnL: 11982.79

          (ourbid) mktbid :: mktask (ourask)
CHECKING
  CSCO -- ( 40.00)  40.10 ::  40.30 ( 40.40)
CHECKING
   ING -- ( 47.40)  47.60 ::  47.70 ( 47.80)
